In [1]:
from itertools import chain
from nltk.tokenize import sent_tokenize
from threading import Thread, Lock
from queue import Queue
import time

from htools import *
from jabberwocky.core import GuiTextChunker
from jabberwocky.speech import Speaker

In [2]:
SPEAKER = Speaker()
CHUNKER = GuiTextChunker(max_chars=70)

In [20]:
text = 'Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you prefer. What is that?'
text_simple = 'A dog is sad. He is alone.'

In [21]:
def stream(text):
    for word in text.split(' '):
        yield word + ' '

In [22]:
def read_response(response, errors=None, hide_on_exit=True):
    print('\nresponse:', response)
    try:
        for sent in sent_tokenize(response):
            for chunk in sent.split('\n\n'):
                SPEAKER.speak(chunk)
                if errors:
                    raise errors[0]
    except RuntimeError:
        pass

In [26]:
def old_concurrent_speaking_typing(streamable, conv_mode=False, pause=.18):
    # Stream function provides "typing" effect.
    threads = []
    errors = []
    full_text = ''
    curr_text = ''
    for chunk in stream(streamable):
        full_text += chunk
        curr_text += chunk
        chunked = CHUNKER.add('response', full_text)
        print(chunked, end='')
        if any(char in chunk for char in ('.', '!', '?', '\n\n')):
            if not errors:
                thread = Thread(target=read_response,
                                args=(curr_text, errors, False))
                thread.start()
                threads.append(thread)
            # Make sure this isn't reset until AFTER the speaker thread starts.
            curr_text = ''
        time.sleep(pause)
    if curr_text and not errors:
        read_response(curr_text)
    for thread in threads: thread.join()

In [27]:
old_concurrent_speaking_typing(text)

'response'
Who 
Who are 
Who are you? 
response: Who are you? 

Who are you? I 
Who are you? I am 
Who are you? I am Mr. 
response: I am Mr. 

Who are you? I am Mr. Nichols. 
response: Nichols. 

Who are you? I am Mr. Nichols. You 
Who are you? I am Mr. Nichols. You can 
Who are you? I am Mr. Nichols. You can call 
Who are you? I am Mr. Nichols. You can call me 
Who are you? I am Mr. Nichols. You can call me John. 
response: You can call me John. 

Who are you? I am Mr. Nichols. You can call me John. Or 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., 
response: Or J.T., 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. 
response: if you prefer. 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What 
Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What 

In [37]:
@coroutine
def read_response_coro():
    # Must send None as an extra last item so that this coroutine knows when 
    # we're done sending in new tokens so it can check for any unread text.
    text = ''
    while True:
        token = yield
        if token is None:
            SPEAKER.speak(sents[0])
        else:
            text += token
            sents = sent_tokenize(text)
            if len(sents) > 1:
                SPEAKER.speak(sents[0])
                text = text.replace(sents[0], '', 1)

In [87]:
# class CoroutinableThread(Thread):
    
#     def __init__(self, target, args=(), kwargs=None):
#         super().__init__(target=target, args=args, kwargs=kwargs)
#         self.target = target(*args, **(kwargs or {}))
        
#     def send(self, val):
#         self.target.send(val)

class CoroutinableThread(Thread):
    
    def __init__(self, target, queue, args=(), kwargs=None):
        super().__init__(target=target, args=args, kwargs=kwargs)
        self.target = target(*args, **(kwargs or {}))
        self.queue = queue
        
    def run(self):
        while True:
            val = self.queue.get()
            self.target.send(val)
            # Must do this after send so our coroutine gets the sentinel.
            if val is None: return

In [88]:
q = Queue()
thread = CoroutinableThread(target=read_response_coro, queue=q, args=())
thread.start()

for t in chain(stream(text_simple), [None]):
    thread.queue.put(t)
    print('LOOP:', t)
    time.sleep(.2)
    
thread.join()

LOOP: A 
LOOP: dog 
LOOP: is 
LOOP: sad. 
LOOP: He 
LOOP: is 
LOOP: alone. 
LOOP: None


In [41]:
# thread = CoroutinableThread(target=read_response_coro, args=())
# thread.start()

# for t in chain(stream(text_simple), [None]):
#     thread.send(t)
#     print(t)
#     time.sleep(.2)

# thread.join()

In [89]:
# def concurrent_speaking_typing(streamable, conv_mode=False, pause=.18):
#     # Stream function provides "typing" effect.
#     threads = []
#     full_text = ''
#     thread = CoroutinableThread(target=read_response_coro, args=())
#     thread.start()
#     for chunk in chain(stream(streamable), [None]):
#         if chunk is not None:
#             full_text += chunk
#             chunked = CHUNKER.add('response', full_text)
#             print(chunked)
#         thread.send(chunk)
#         time.sleep(pause)
#     thread.join()

def concurrent_speaking_typing(streamable, conv_mode=False, pause=.18):
    # Stream function provides "typing" effect.
    full_text = ''
    q = Queue()
    thread = CoroutinableThread(target=read_response_coro, queue=q, args=())
    thread.start()
    for chunk in stream(streamable):
        if conv_mode:
            chunked = CHUNKER.add(
                'conv_transcribed',
                CONV_MANAGER.full_conversation(include_summary=False)
            )
        else:
            chunked = CHUNKER.add('response', full_text)
            full_text += chunk
            print(chunked)
        thread.queue.put(chunk)
        time.sleep(pause)
    thread.queue.put(None)
    thread.join()
#     hide_item(data['interrupt_id'])
#     hide_item(data['query_msg_id'])

In [90]:
concurrent_speaking_typing(text)


Who 

Who are 

Who are you? 

Who are you? I 

Who are you? I am 

Who are you? I am Mr. 

Who are you? I am Mr. Nichols. 

Who are you? I am Mr. Nichols. You 

Who are you? I am Mr. Nichols. You can 

Who are you? I am Mr. Nichols. You can call 

Who are you? I am Mr. Nichols. You can call me 

Who are you? I am Mr. Nichols. You can call me John. 

Who are you? I am Mr. Nichols. You can call me John. Or 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What is 

Who are you? I am Mr. Nichols. You can call me John. Or J.T., if you
prefer. What is that? 


## Threading scratch

Question: can we start a thread within a thread?

In [9]:
from datetime import datetime as dt

from htools import *

In [20]:
def foo():
    start = time.time()
    while True:
        print(dt.now().strftime('%Y-%m-%d %H:%M:%S'))
        time.sleep(1)
        if time.time() - start >= 5: break

In [21]:
def targ():
    thread = Thread(target=foo, args=())
    thread.start()
    res = [i for i in sleepy_range(10, wait=.5)]
    thread.join()
    return res

In [22]:
res = targ()

2021-08-18 20:53:43
2021-08-18 20:53:44
2021-08-18 20:53:45
2021-08-18 20:53:46
2021-08-18 20:53:47


In [23]:
res

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [25]:
thread = Thread(target=targ, args=())
thread.start()
main_res = [i for i in sleepy_range(5, wait=.1)]
thread.join()

2021-08-18 20:55:12
2021-08-18 20:55:13
2021-08-18 20:55:14
2021-08-18 20:55:15
2021-08-18 20:55:16


In [26]:
main_res

[0, 1, 2, 3, 4]

**Conclusion**

Looks like yes.